## Keras序贯模型(Sequential Models)

In [7]:
# sequential models
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D

def CNN_model():
    model = Sequential()
    model.add(Conv1D(64, 3, activation='relu', input_shape=(20, 100)))
    model.add(Conv1D(64, 3, activation='relu'))
    model.add(MaxPooling1D(3))
    model.add(Conv1D(128, 3, activation='relu'))
    model.add(Conv1D(128, 3, activation='relu'))
    model.add(GlobalAveragePooling1D())
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
    model.summary()
    return model
model=CNN_model()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_7 (Conv1D)            (None, 18, 64)            19264     
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 16, 64)            12352     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 5, 64)             0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 3, 128)            24704     
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 1, 128)            49280     
_________________________________________________________________
global_average_pooling1d_1 ( (None, 128)               0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)              

## Keras函数式(functional)API构建模型

In [22]:
# 普通模型
# 与Sequential模型不同，你必须创建独立的Input层物件的instance并定义输入数据张量的维度形状(tensor shape)。
# 输入层采用一个张量形状参数(tensor shape),它是一个tuple,用于宣告输入张量的维度。
# 例如：我们要把MNIST的每张图像(28 * 28)打平成一个一维(784)的张量作为一个多层感知机(MLP)的Input。

from keras.layers import Input, Dense
from keras.models import Model

# 层对象接受张量为参数，返回一个张量。
# 输入是张量，输出也是张量的一个框架就是一个模型，通过Model定义。

def model():
    inputs = Input(shape=(784, ))
    x = Dense(64, activation='relu')(inputs)
    x = Dense(64, activation='relu')(x)
    outputs = Dense(10, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.summary()  # 画出模型参数


model = model()

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 784)               0         
_________________________________________________________________
dense_30 (Dense)             (None, 64)                50240     
_________________________________________________________________
dense_31 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_32 (Dense)             (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


### 共享层模型

In [25]:
# 共享输入层
from keras.models import Model
from keras.layers import Input, Dense, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPool2D
from keras.layers.merge import concatenate
from keras.utils import plot_model


# 该模型使用28　×　28　像素的灰阶图像。有两个CNN特征提取子模型共享这个输入；第一个具有４的内核大小和第二个８的内核大小。这些特征提取子模型的输出被平坦化(flatten)为向量(vector),并且被串成一个长向量；然后被传递到完全连接的层以用于最终输出层之前进行10类别预测。
def cnn_inouts_shared():
    
    inputs = Input(shape=(28, 28, 1))
    # 第一个特征提取层
    conv1 = Conv2D(32, kernel_size=4, activation='relu')(inputs)# <- 看这里
    pool1 = MaxPool2D(pool_size=(2, 2))(conv1)
    flat1 = Flatten()(pool1)
    
    # 第二个特征提取层
    conv2 = Conv2D(16,kernel_size=8,activation='relu')(inputs) # <- 看这里
    pool2 = MaxPool2D(pool_size=(2,2))(conv2)
    flat2 = Flatten()(pool2)
    
    # 把这两个特征提取层的结果拼接起来
    merge = concatenate([flat1,flat2])
    
    # 进行全连接层
    hidden1 = Dense(64,activation='relu')(merge)
    # 输出层
    output = Dense(10,activation='softmax')(hidden1)
    model = Model(inputs =inputs,outputs = output)
    # 打印网络结构
    model.summary()
    
cnn_model =cnn_inouts_shared()

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 25, 25, 32)   544         input_10[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 21, 21, 16)   1040        input_10[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 12, 12, 32)   0           conv2d_1[0][0]                   
___________________________________________________________________________________________

In [27]:
# 共享特征提取层
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers.recurrent import LSTM
from keras.layers.merge import concatenate

# 该模型的输入是一个特征的784个时间步长。具有１０个存储单元的LSTM层解释这个序列。第一种解释模型是浅层单连通层，第二层是深层３层模型。两个解释模型的输出连接成一个长向量，传递给用于进行１０类别分类预测的输出层。
def lstm_feature_shared():
    inputs = Input(shape=(784, 1))

    # 特征提取层
    extract1 = LSTM(128)(inputs)

    # 第一个解释层(浅层单连通层)
    inter1 = Dense(10, activation='relu')(extract1)  # <- 看这里
    # 第二个解释层(深层3层模型)
    inter21 = Dense(64, activation='relu')(extract1)  # <- 看这里
    inter22 = Dense(32, activation='relu')(inter21)
    inter23 = Dense(10, activation='relu')(inter22)
    
    # 把两个特征提取层的结果拼起来
    merge=concatenate([inter1,inter23],name ='merge')
    
    # 输出层
    output = Dense(10,activation='softmax',name='output')(merge)
    
    model=Model(inputs=inputs,outputs=output)
    model.summary()
    
lstm_model=lstm_feature_shared()

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 784, 1)       0                                            
__________________________________________________________________________________________________
lstm_2 (LSTM)                   (None, 128)          66560       input_12[0][0]                   
__________________________________________________________________________________________________
dense_36 (Dense)                (None, 64)           8256        lstm_2[0][0]                     
__________________________________________________________________________________________________
dense_37 (Dense)                (None, 32)           2080        dense_36[0][0]                   
___________________________________________________________________________________________

In [32]:
# 多输入模型
from keras.models import Model
from keras.layers import Input,Dense,Flatten
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers.merge import concatenate
    
#我们将开发一个图像分类模型，将图像的两个版本作为输入，每个图像的大小不同。
#特别是一个灰阶的64 * 64版本和一个32 * 32 彩色版本。
#分离的特征提取CNN模型对每个模型进行操作，然后将两个模型的结果连接起来进行解释和最终的预测。

#请注意在创建Model()实例(instance)时，我们将两个输入层定义为一个数组(array)。

def lstm_multi_inpust():
    
    # 第一个输入层
    img_gray_input = Input(shape=(64,64,1),name='img_gray_input')
    conv11=Conv2D(32,kernel_size=4,activation='relu')(img_gray_input)
    pool11 = MaxPool2D(pool_size=(2,2),name='pool11')(conv11)
    conv12 = Conv2D(16,kernel_size=4,activation='relu',name='conv12')(pool11)
    pool12 = MaxPool2D(pool_size=(2,2),name='pool12')(conv12)
    flat1 = Flatten()(pool12)
    
    # 第二个输入层
    img_rgb_input = Input(shape=(32,32,3),name='img_rgb_input')
    conv21 = Conv2D(32,kernel_size=4,activation='relu',name='conv21')(img_rgb_input)
    pool21 = MaxPool2D(pool_size=(2,2),name='pool21')(conv21)
    conv22 = Conv2D(16,kernel_size=4,activation='relu',name='conv22')(pool21)
    pool22 = MaxPool2D(pool_size=(2,2),name='pool22')(conv22)
    flat2 = Flatten()(pool22)
    
    # 把两个特征提取层的结果拼起来
    merge = concatenate([flat1,flat2])
    
    # 用隐藏的全连接层来解释特征
    hidden1 = Dense(128,activation='relu',name='hidden1')(merge)
    hidden2 = Dense(64,activation='relu',name='hidden2')(hidden1)

    # 输出层
    output = Dense(10,activation='softmax',name='output')(hidden2)
    
    model=Model(inputs=[img_gray_input,img_rgb_input],outputs=output)
    
    model.summary()

multi_input_model = lstm_multi_inpust()

Model: "model_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img_gray_input (InputLayer)     (None, 64, 64, 1)    0                                            
__________________________________________________________________________________________________
img_rgb_input (InputLayer)      (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 61, 61, 32)   544         img_gray_input[0][0]             
__________________________________________________________________________________________________
conv21 (Conv2D)                 (None, 29, 29, 32)   1568        img_rgb_input[0][0]              
___________________________________________________________________________________________

In [36]:
# keras中的TimeDistributed的作用解释
# rnn中，输入向量维度为：[batch, timesteps,sequences] , 假设为[100, 20,10]。
# 每一个时间步都会输出一个output_nodes大小的向量，即当输入为[100,20,10]时候，输出为[100,20,5], 
# 但是当去return_sequences=false时，就会只显示最后一个时间步的输出,省略中间的一个维度，为[100,5]
# 而keras的batchs在显示中为None,只需要在model.fit中注明batchs为多少就好。

# 上面讲到当return_sequences为True时，每一步都会返回一个output_nodes大小的向量：[,20,5]
output_nodes = 5
model.add(LSTM(output_nodes,input_shape=(timesteps,sequences),,return_sequences=True))
# [,20,5]
model.add(TimeDistributed(Dense(1))) 
# TimeDistributed对每个时间步的数据都做一个相同的操作，比较方便保持数据的维度。
# [,20,1]


# TimeDistributed层在每个时间步上均操作了Dense
# 如果你使用正常的Dense层，你最后只会得到一个结果,下面的output2也会变成（None，10）

SyntaxError: invalid syntax (<ipython-input-36-4b14f60fba97>, line 9)

In [35]:
# 多输出模型
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed
# 我们将开发一个模型，进行两种不同模型的预测。
#给定一个特征的784个时间步长的输入序列，该模型将对该序列进行分类并输出具有相同长度的新序列。
# LSTM层解释输入序列并返回每个时间步长的隐藏状态。
#第一个输出模型创建一个堆叠的LSTM，解释这些特征，并进行多类别预测。
#第二个输出模型使用相同的输出层对每个输入时间步进行多类别预测。
#lstm：[batch, timesteps,sequences],batch不输入到input_shape中

def lstm_multi_output():
    # 输入层
    mnist_input = Input(shape=(784, 1), name='input') 
    # 把每一个像素想成是一序列有前后关系的time_steps
    
    # 特征提取层
    extract = LSTM(64, return_sequences=True, name='extract')(mnist_input)
    # 分类输出
    class11 = LSTM(32,name='class11')(extract) #默认 return_sequences=False，就不输出784个时间步
    class12 = Dense(32,activation='relu',name='class12')(class11)
    output1 = Dense(10,activation='softmax',name= 'output1')(class12)
    
    # 序列输出
    output2 = TimeDistributed(Dense(10,activation='softmax'),name= 'output2')(extract)
    # TimeDistributed 来将 Dense 层独立地应用到 这 784个时间步的每一个，保持数据的维度
    # 以Model来组合整个网络
    model = Model(inputs=mnist_input,outputs=[output1,output2])
    model.summary()
    
multi_output_model = lstm_multi_output()


Model: "model_14"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 784, 1)       0                                            
__________________________________________________________________________________________________
extract (LSTM)                  (None, 784, 64)      16896       input[0][0]                      
__________________________________________________________________________________________________
class11 (LSTM)                  (None, 32)           12416       extract[0][0]                    
__________________________________________________________________________________________________
class12 (Dense)                 (None, 32)           1056        class11[0][0]                    
___________________________________________________________________________________________

In [48]:
# 自己的练习，多个输入：
from keras.layers import Dense, Input, LSTM, Dropout,ReLU,Bidirectional
from keras.optimizers import Adam
def BilstmCNN():
    inputs = Input(shape=(50, 128), name='in')
    # 提取特
    conv_list = list()
    for i in range(2):
        conv1 = Conv1D(128, i + 1)(inputs)
        bn = BatchNormalization()(conv1)
        activation = ReLU()(bn)
        x = MaxPool1D()(activation)
        x = Dropout(0.5)(x)
        conv_list.append(x)
    extract = concatenate(conv_list, axis=1, name='extract')

    # 事件检测输出层
    lstm1 = Bidirectional(LSTM(128, return_sequences=False, dropout=0.35),
                          merge_mode="concat",
                          name='lsmt1')(extract)
    x = Dense(64)(lstm1)
    x = Dropout(0.3)(x)
    output1 = Dense(2, activation='softmax', name='output1')(x)

    # 事件分类输出层
    lstm2 = Bidirectional(LSTM(128, return_sequences=False, dropout=0.35),
                          merge_mode="concat",
                          name='lsmt2')(extract)
    x = Dense(64)(lstm2)
    x = Dropout(0.3)(x)
    output2 = Dense(11, activation='softmax', name='output2')(x)

    model = Model(inputs=inputs, outputs=[output1, output2])
    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(0.0001),
                  metrics=['accuracy'])
    model.summary()
    return model
model = BilstmCNN() 

Model: "model_18"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
in (InputLayer)                 (None, 50, 128)      0                                            
__________________________________________________________________________________________________
conv1d_25 (Conv1D)              (None, 50, 128)      16512       in[0][0]                         
__________________________________________________________________________________________________
conv1d_26 (Conv1D)              (None, 49, 128)      32896       in[0][0]                         
__________________________________________________________________________________________________
batch_normalization_17 (BatchNo (None, 50, 128)      512         conv1d_25[0][0]                  
___________________________________________________________________________________________